#  Entregable 4 - Aprendizaje Automático II


***<p style="text-align:center;">Implementación de una MLP con PyTorch</p>***

En este cuaderno vamos a implementar una MLP (Multi-Layer Perceptron), esta vez, ayudándonos de `PyTorch`.

### Normas de Entrega

1. El formato de entrega será en una carpeta comprimida con nombre: {Iniciales de Nombre y Apellidos}_E4.zip, en Aula Virtual en la fecha señalada en la plataforma y comunicada en clase previamente.
    * Por ejemplo: Iván Ramírez Díaz ==> `IRD_E4.zip`
2. El contenido de dicha carpeta será:
    * Obligatorio: Notebook relleno del Entregable 4.
    * Opcional: Memoria (pdf), en caso de necesitar dar alguna explicación.
3. Antes de la entrega, se debe comprobar que el código completo funciona.
4. La entrega es individual.

### Evaluación

La práctica entregable tiene un peso global de 1/4 puntos (los 4 entregables son el 10% de la nota final).

La práctica entregable se calificará sobre 10 puntos. Las puntuaciones son las siguientes:

- **[Ejercicio 1]** Implementa y entrena el modelo `MLP` usando `PyTorch`. (2 puntos)
- **[Ejercicio 2]** Implementa y entrena el modelo `MLP` usando `Sequential` de `PyTorch`. (1 punto)
- **[Ejercicio 3]** Entrena haciendo uso de `GPU` o `TPU` y compara con el tiempo de entrenamiento en `CPU`. (1 punto)
- **[Ejercicio 4]** Crea un dataset de `PyTorch` (`Dataset`) a partir de los datos de `breast cancer dataset` y entrena varias MLPs usando la entropía cruzada binaria. (3 puntos)
- **[Ejercicio 5]** Repite todo el proceso ahora para el `Iris` dataset multi-clase. (3 puntos)


## **[Ejercicio 1]**

Con los mismos datos del Entregable 3, que se proporcionan a continuación:

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

np.random.seed(1)

# Parámetros de entrada y salida
input_dim = 2   # Dimensiones de entrada
output_dim = 1  # Dimensiones de salida

# Generación de datos de ejemplo
n_samples = 250
X = np.random.rand(n_samples, input_dim)
y = np.sin(2 * np.pi * X[:,0:1]) + np.cos(2 * np.pi * X[:,1:]) + 0.1 * np.random.randn(n_samples, output_dim)

Haz un split de `train` (70%), `valid` (15%) y `test` (15%):

In [9]:
######################### COMPLETAR ############################
from sklearn.model_selection import train_test_split

# Dividimos primero en entrenamiento y conjunto temporal de validación+test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Dividimos el conjunto temporal en validación y test
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Tamaños de los conjuntos resultantes
print("Tamaño del conjunto de entrenamiento:", X_train.shape[0])
print("Tamaño del conjunto de validación:", X_valid.shape[0])
print("Tamaño del conjunto de prueba:", X_test.shape[0])
###################### FIN COMPLETAR ###########################

Tamaño del conjunto de entrenamiento: 175
Tamaño del conjunto de validación: 37
Tamaño del conjunto de prueba: 38


Crea un modelo MLP en `PyTorch` de forma similar a como lo hiciste en el Entregable 3:

In [10]:
########################### COMPLETAR ###########################
# Importa lo que necesites
import torch
import torch.nn as nn

######################### FIN COMPLETAR #########################

# Definir el modelo MLP
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        ########################### COMPLETAR ###########################
        # Capa lineal con 512 neuronas de salida
        self.F1 = nn.Linear(input_dim, 512)
        # Capa de activación
        self.Act1 = nn.ReLU()
        # Capa lineal con 1024 neuronas de salida
        self.F2 = nn.Linear(512, 1024)
        # Capa de activación
        self.Act2 = nn.ReLU()
        # Capa de salida
        self.F3 = nn.Linear(1024, output_dim)
        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        x = self.F1(x)
        x = self.Act1(x)
        x = self.F2(x)
        x = self.Act2(x)
        x = self.F3(x)
        return x
        ######################### FIN COMPLETAR #########################

# Inicializar el modelo
input_dim = 2  # Dimensión de entrada
output_dim = 1  # Dimensión de salida

model = MLP(input_dim, output_dim)
print(model)

MLP(
  (F1): Linear(in_features=2, out_features=512, bias=True)
  (Act1): ReLU()
  (F2): Linear(in_features=512, out_features=1024, bias=True)
  (Act2): ReLU()
  (F3): Linear(in_features=1024, out_features=1, bias=True)
)


Entrena el modelo con datos anteriores:

In [11]:
########################### COMPLETAR ###########################
# Convertir datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32)

# Inicializar el modelo
model = MLP(input_dim, output_dim)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss()   # Pérdida para regresión
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)   # Optimizador

######################### FIN COMPLETAR #########################

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 32

# Entrenamiento

for epoch in range(n_epochs):
    model.train()  # Modo entrenamiento
    permutation = torch.randperm(X_train_tensor.size(0))  # Mezclar datos

    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]

        ########################### COMPLETAR ###########################
        # Selecciona los batches aleatoriamente con `indices` dado
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        # Forward pass
        predictions = model(batch_X)         #calculamos las predaciones del modelo
        loss = loss_fn(predictions, batch_y) #calculamos la perdida del batch

        # Backward pass y optimización
        optimizer.zero_grad()               #limpiamos los gradientes previos
        loss.backward()                     #calculamos los gradientes en el backward
        optimizer.step()                    #ajustamos los pesos según los gradientes calculados
        ######################### FIN COMPLETAR #########################

    ########################### COMPLETAR ###########################
    # Validación cada 500 épocas
    if (epoch + 1) % 500 == 0:
        # Modo evaluación
        model.eval()
        with torch.no_grad():
            val_predictions = model(X_valid_tensor)
            val_loss = loss_fn(val_predictions, y_valid_tensor)
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    ######################### FIN COMPLETAR #########################


Epoch 500/5000, Loss: 0.0224, Val Loss: 0.0342
Epoch 1000/5000, Loss: 0.0183, Val Loss: 0.0358
Epoch 1500/5000, Loss: 0.0116, Val Loss: 0.0211
Epoch 2000/5000, Loss: 0.0139, Val Loss: 0.0202
Epoch 2500/5000, Loss: 0.0111, Val Loss: 0.0270
Epoch 3000/5000, Loss: 0.0118, Val Loss: 0.0198
Epoch 3500/5000, Loss: 0.0077, Val Loss: 0.0197
Epoch 4000/5000, Loss: 0.0056, Val Loss: 0.0228
Epoch 4500/5000, Loss: 0.0139, Val Loss: 0.0309
Epoch 5000/5000, Loss: 0.0050, Val Loss: 0.0205


Una vez entrenado, podemos comprobar el nivel de ajuste con los datos de entrenamiento:

In [12]:
import plotly.graph_objs as go

# Prediciones de entrenamiento
predictions = model(X_train_tensor).detach().numpy()

# Plots
fig = go.Figure()

# Plot predicciones
fig.add_trace(go.Scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=predictions[:, 0],
    mode='markers',
    marker=dict(color='red', size=8, symbol='circle', opacity=0.8),
    name='y_pred',
))

# Plot y_train
fig.add_trace(go.Scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=y_train.flatten(),
    mode='markers',
    marker=dict(color='blue', size=8, symbol='circle', opacity=0.8),
    name='y_true',
))

# Ajuste del layout
fig.update_layout(
    scene=dict(
        xaxis_title='X1',
        yaxis_title='X2',
        aspectmode='cube',  # Ensures equal scaling of all axes
        xaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Light gray background
        yaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Light gray background
        zaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True)   # Light gray background
    ),
    width=800,  # Increase the size of the figure
    height=800,  # Increase the size of the figure
    margin=dict(l=0, r=0, b=0, t=0),  # Reduce margins
    title="Predición de la MLP",  # Add title
    title_font=dict(size=20, family='Arial, sans-serif'),  # Customize title font
    scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),  # Set the default camera angle for better visualization
)

fig.show()

## **[Ejercicio 2]**

Repite el proceso, ahora, reescribiendo la clase `MLP` (llámala `MLPSeq`) haciendo uso de `Sequential`.

Ayúdate de la documentación: https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definir el modelo MLP
class MLPSeq(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLPSeq, self).__init__()
        self.model = nn.Sequential(
        ########################### COMPLETAR ###########################
             # Capa lineal con 512 neuronas de salida
            nn.Linear(input_dim, 512),
             # Capa de activación
            nn.ReLU(),
             # Capa lineal con 1024 neuronas de salida
            nn.Linear(512, 1024),
             # Capa de activación
            nn.ReLU(),
             # Capa de salida
            nn.Linear(1024, output_dim)
        ######################### FIN COMPLETAR #########################
        )

    def forward(self, x):
        ########################### COMPLETAR ###########################
        return self.model(x)
        ######################### FIN COMPLETAR #########################

# Inicializar el modelo
input_dim = 2  # Dimensión de entrada
output_dim = 1  # Dimensión de salida

model = MLPSeq(input_dim, output_dim)
print(model)


MLPSeq(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=1, bias=True)
  )
)


Entrena de nuevo y compara los resultados:

In [14]:
########################### COMPLETAR ###########################
# Entrena de nuevo con MLPSeq y compara los resultados:

# Convertir datos a tensores de PyTorch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32)

# Inicializar el modelo
model = MLPSeq(input_dim, output_dim)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss() # Pérdida para regresión
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01) # Optimizador SGD

######################### FIN COMPLETAR #########################

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 32

# Entrenamiento

for epoch in range(n_epochs):
    model.train()  # Modo entrenamiento
    permutation = torch.randperm(X_train_tensor.size(0))  # Mezclar datos

    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]

        ########################### COMPLETAR ###########################
        # Selecciona los batches aleatoriamente con `indices` dado
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        # Forward pass
        predictions = model(batch_X)
        loss = loss_fn(predictions, batch_y)

        # Backward pass y optimización
        optimizer.zero_grad() # Hacer antes del .backward() o después del .step()
        loss.backward()
        optimizer.step()
        ######################### FIN COMPLETAR #########################

    ########################### COMPLETAR ###########################
    # Validación cada 500 épocas
    if (epoch + 1) % 500 == 0:
        model.eval() # Modo evaluación
        with torch.no_grad():
            val_predictions = model(X_valid_tensor)
            val_loss = loss_fn(val_predictions, y_valid_tensor)
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    ######################### FIN COMPLETAR #########################


Epoch 500/5000, Loss: 0.0148, Val Loss: 0.0284
Epoch 1000/5000, Loss: 0.0138, Val Loss: 0.0221
Epoch 1500/5000, Loss: 0.0109, Val Loss: 0.0245
Epoch 2000/5000, Loss: 0.0082, Val Loss: 0.0199
Epoch 2500/5000, Loss: 0.0087, Val Loss: 0.0217
Epoch 3000/5000, Loss: 0.0048, Val Loss: 0.0221
Epoch 3500/5000, Loss: 0.0073, Val Loss: 0.0220
Epoch 4000/5000, Loss: 0.0084, Val Loss: 0.0190
Epoch 4500/5000, Loss: 0.0124, Val Loss: 0.0300
Epoch 5000/5000, Loss: 0.0083, Val Loss: 0.0206


## **[Ejercicio 3]**

Para comparar los tiempos de entrenamiento, primero vamos a implementar el entrenamiento como función.

Define una función `train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size)`

In [15]:
def train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size):
    ########################### COMPLETAR ###########################

    for epoch in range(n_epochs):
        model.train()

        #barajamos los índices de los datos de train
        permutation = torch.randperm(X_train_tensor.size(0))

        #entrenamos en mini-batches
        for i in range(0, X_train_tensor.size(0), batch_size):
            #seleccionamos los índiuces del batch
            indices = permutation[i:i + batch_size]

            #extraemos batch_x y batch_y
            batch_X = X_train_tensor[indices]
            batch_y = y_train_tensor[indices]

            #FORDWARD
            y_pred = model(batch_X)

            #PERDIDA
            loss = loss_fn(y_pred, batch_y)

            #BACKWARD
            optimizer.zero_grad() #limpiamos los gradientes anteriores
            loss.backward()
            optimizer.step() #actualizamos los pesos


        # VALIDACION cada 500 épocas
        if (epoch +1) % 500 == 0:
            model.eval()
            with torch.no_grad():
                val_pred = model(X_valid_tensor)
                val_loss = loss_fn(val_pred, y_valid_tensor)
                print(f"Epoch {epoch+1}/{n_epochs} - Loss={loss.item():.4f}, Val Loss={val_loss.item():.4f}")


    ######################### FIN COMPLETAR #########################


Entrena el mismo modelo en `CPU` y en `GPU` para distintos valores del batch. ¿Qué conclusiones sacas?

In [16]:
import time

start = time.time()
########################### COMPLETAR ###########################
# Convertir datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32)

# Inicializar el modelo
model = MLPSeq(input_dim=2, output_dim=1)

# Definir la función de pérdida y el optimizador
loss_fn =  nn.MSELoss() # Pérdida para regresión
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01) # Optimizador SGD

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 32

train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size)
######################### FIN COMPLETAR #########################

end = time.time()
print('CPU time: {}'.format(end - start))


Epoch 500/5000 - Loss=0.0309, Val Loss=0.0406
Epoch 1000/5000 - Loss=0.0121, Val Loss=0.0222
Epoch 1500/5000 - Loss=0.0379, Val Loss=0.0448
Epoch 2000/5000 - Loss=0.0161, Val Loss=0.0302
Epoch 2500/5000 - Loss=0.0055, Val Loss=0.0212
Epoch 3000/5000 - Loss=0.0105, Val Loss=0.0219
Epoch 3500/5000 - Loss=0.0065, Val Loss=0.0217
Epoch 4000/5000 - Loss=0.0077, Val Loss=0.0232
Epoch 4500/5000 - Loss=0.0078, Val Loss=0.0200
Epoch 5000/5000 - Loss=0.0087, Val Loss=0.0195
CPU time: 67.67757248878479


In [17]:
start = time.time()

########################### COMPLETAR ###########################
# Convertir datos a tensores de PyTorch y pasarlos a GPU
device = 'cuda'

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32).to(device)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32).to(device)

# Inicializar el modelo
model = MLPSeq(input_dim=2, output_dim=1).to(device)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss() # Pérdida para regresión
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # Optimizador SGD

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 32

train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size)
######################### FIN COMPLETAR #########################

end = time.time()
print('GPU time: {}'.format(end - start))

Epoch 500/5000 - Loss=0.0408, Val Loss=0.0342
Epoch 1000/5000 - Loss=0.0071, Val Loss=0.0289
Epoch 1500/5000 - Loss=0.0094, Val Loss=0.0211
Epoch 2000/5000 - Loss=0.0156, Val Loss=0.0218
Epoch 2500/5000 - Loss=0.0086, Val Loss=0.0200
Epoch 3000/5000 - Loss=0.0119, Val Loss=0.0244
Epoch 3500/5000 - Loss=0.0082, Val Loss=0.0193
Epoch 4000/5000 - Loss=0.0132, Val Loss=0.0263
Epoch 4500/5000 - Loss=0.0081, Val Loss=0.0206
Epoch 5000/5000 - Loss=0.0072, Val Loss=0.0250
GPU time: 38.900794506073


## **[Ejercicio 4]**

Crea, a partir del `breast_cancer_dataset` que ya conoces, un dataset de `PyTorch`. Crea su correspondiente dataloader para train, valid, y test:

In [18]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Carga datos y etiquetas
cancer = load_breast_cancer()
data, labels = cancer.data, cancer.target

# Crea el dataset
class BreastCancerDataset(Dataset):
    ########################### COMPLETAR ###########################
    def __init__(self, data, labels):
        """
        Constructor del Dataset que almacena los datos y las etiquetas.
        """
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32).view(-1,1)



    def __len__(self):
        """
        Devuelve el tamaño del Dataset.
        """
        return len(self.data)


    def __getitem__(self, idx):
        """
        Devuelve un dato (features) y su etiqueta correspondiente.
        """
        return self.data[idx], self.labels[idx]
        ######################### FIN COMPLETAR #########################

########################### COMPLETAR ###########################
# Dividir el dataset en train (70%), valid (15%) y test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Crear los datasets para train, valid y test
train_dataset = BreastCancerDataset(X_train, y_train)
valid_dataset = BreastCancerDataset(X_valid, y_valid)
test_dataset = BreastCancerDataset(X_test, y_test)

# Crear los DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)  # Para entrenamiento, mezclar los datos
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False) # Para validación, no mezclar los datos
test_loader =  DataLoader(test_dataset, batch_size = batch_size, shuffle = False) # Para test, no mezclar los datos
######################### FIN COMPLETAR #########################

for batch_idx, (batch_data, batch_labels) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print(f"Datos: {batch_data.shape}")  # Dimensiones del batch de datos
    print(f"Etiquetas: {batch_labels.shape}")  # Dimensiones del batch de etiquetas

Batch 1:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 2:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 3:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 4:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 5:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 6:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 7:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 8:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 9:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 10:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 11:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 12:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32, 1])
Batch 13:
Datos: torch.Size([14, 30])
Etiquetas: torch.Size([14, 1])


Crea una función de entrenamiento `train_model` y otra de evaluación `evaluate_model`:

In [34]:
# Función de entrenamiento y validación
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, n_epochs=n_epochs, device=device):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a `device`
                # Completar
    model = model.to(device)
    for epoch in range(n_epochs):
        # Modelo en modo entrenamiento
                # Cpmpletar
        model.train()

        train_loss = 0.0
        for features, labels in train_loader:

            # Enviar datos a 'device
            features = features.to(device)
            labels = labels.to(device).view(-1,1)  # Ajustar dimensiones y enviar a GPU

            # Forward
            outputs = model(features)
            loss = loss_fn(outputs, labels)

            # Backward y optimización
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_loss += loss.item() * features.size(0)

        train_loss /= len(train_loader.dataset)

        # Modelo en modo validación
                # Completar

        valid_loss = 0.0
        with torch.no_grad():
            for features, labels in valid_loader:
                features = features.to(device)
                labels = labels.to(device).view(-1,1)

                outputs = model(features)
                loss = loss_fn(outputs, labels)
                valid_loss += loss.item() * features.size(0)
        valid_loss /= len(valid_loader.dataset)

        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
    ######################### FIN COMPLETAR #########################

# Función para evaluar el modelo
def evaluate_model(model, test_loader, device=device):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a `device`
        # Completar
    model = model.to(device)
    # Modelo en modo validación
        # Completar
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad(): # No hace falta calcular gradientes en evaluación. Así, evitamos cómputos innecesarios
        for features, labels in test_loader:
            # Enviar datos a 'device
            features = features.to(device)
            labels = labels.to(device).view(-1,1)
            # Predicciones
            outputs = model(features)
            predictions =  (outputs >= 0.5).float() # Umbral de clasificación
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy
    print(f"Test Accuracy: {accuracy:.4f}")


Crea tres modelos de MLP:
1. MLP_s (pequeña): 1 capa oculta.
2. MPL_n (mediana): 2 capas ocultas.
3. MPL_b (grande): 5 capas ocultas.

In [35]:
# Definir el modelo MLP_s
class MLP_s(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP_s, self).__init__()
        ########################### COMPLETAR ###########################
        F1 = nn.Linear(input_dim, 128)  # Capa lineal con 128 neuronas de salida
        Act1 = nn.ReLU()  # Capa de activación
        F2 =  nn.Linear(128, output_dim) # Capa de salida

        self.F1 = F1
        self.Act1 = Act1
        self.F2 = F2

        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        x = self.F1(x)
        x = self.Act1(x)
        x = self.F2(x)
        return x
        ######################### FIN COMPLETAR #########################

# Definir el modelo MLP_m
class MLP_m(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP_m, self).__init__()
        ########################### COMPLETAR ###########################
        F1 = nn.Linear(input_dim, 128) # Capa lineal con 128 neuronas de salida
        Act1 = nn.ReLU() # Capa de activación
        F2 = nn.Linear(128, 64) # Capa lineal con 64 neuronas de salida
        Act2 = nn.ReLU() # Capa de activación
        F3 = nn.Linear(64, output_dim)  # Capa de salida

        self.F1 = F1
        self.Act1 = Act1
        self.F2 = F2
        self.Act2 = Act2
        self.F3 = F3

        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        x = self.F1(x)
        x = self.Act1(x)
        x = self.F2(x)
        x = self.Act2(x)
        x = self.F3(x)
        return x
        ######################### FIN COMPLETAR #########################

# Definir el modelo MLP_b
class MLP_b(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP_b, self).__init__()
        ########################### COMPLETAR ###########################
        F1 = nn.Linear(input_dim, 128)   # Capa lineal con 128 neuronas de salida
        Act1 = nn.ReLU() # Capa de activación
        F2 = nn.Linear(128, 128)   # Capa lineal con 128 neuronas de salida
        Act2 = nn.ReLU() # Capa de activación
        F3 = nn.Linear(128,128)   # Capa lineal con 128 neuronas de salida
        Act3 = nn.ReLU() # Capa de activación
        F4 = nn.Linear(128, 128)   # Capa lineal con 128 neuronas de salida
        Act4 = nn.ReLU() # Capa de activación
        F5 =  nn.Linear(128,128)  # Capa lineal con 128 neuronas de salida
        Act5 = nn.ReLU() # Capa de activación
        F6 = nn.Linear(128, output_dim)   # Capa de salida

        self.F1 = F1
        self.Act1 = Act1
        self.F2 = F2
        self.Act2 = Act2
        self.F3 = F3
        self.Act3 = Act3
        self.F4 = F4
        self.Act4 = Act4
        self.F5 = F5
        self.Act5 = Act5
        self.F6 = F6


        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        x = self.F1(x)
        x = self.Act1(x)
        x = self.F2(x)
        x = self.Act2(x)
        x = self.F3(x)
        x = self.Act3(x)
        x = self.F4(x)
        x = self.Act4(x)
        x = self.F5(x)
        x = self.Act5(x)
        x = self.F6(x)
        return x
        ######################### FIN COMPLETAR #########################

Entrena y evalúa:

In [36]:
########################### COMPLETAR ###########################

# Inicializar el modelo
# Parámetros de entrada y salida
input_dim = 30   # Dimensiones de entrada
output_dim = 1  # Dimensiones de salida

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Detectar GPU

model_s = MLP_s(input_dim, output_dim).to(device)
model_m = MLP_m(input_dim, output_dim).to(device)
model_b = MLP_b(input_dim, output_dim).to(device)

# Definir función de pérdida y optimizador
loss_fn = nn.BCEWithLogitsLoss()  # Ojo con la última capa de activación del modelo
optimizer_s = optim.Adam(model_s.parameters(), lr=0.001)
optimizer_m = optim.Adam(model_m.parameters(), lr=0.001)
optimizer_b = optim.Adam(model_b.parameters(), lr=0.001)

n_epochs = 5

# Entrenar el modelo_s
train_model(model_s, train_loader, valid_loader, loss_fn, optimizer_s, n_epochs, device)

# Evaluar el modelo_s
acc_model_s = evaluate_model(model_s, test_loader, device)

# Entrenar el modelo_m
train_model(model_m, train_loader, valid_loader, loss_fn, optimizer_m, n_epochs, device)

# Evaluar el modelo_m
acc_model_m = evaluate_model(model_m, test_loader, device)

# Entrenar el modelo_b
train_model(model_b, train_loader, valid_loader, loss_fn, optimizer_b, n_epochs, device)

# Evaluar el modelo_b
acc_model_b = evaluate_model(model_b, test_loader, device)

print(f"Test Accuracies. Model s: {acc_model_s:.4f} | Model m: {acc_model_m:.4f} | Model b: {acc_model_b:.4f}")

######################### FIN COMPLETAR #########################


Epoch 1/5, Train Loss: 4.8308, Valid Loss: 2.7492
Epoch 2/5, Train Loss: 2.3588, Valid Loss: 1.0327
Epoch 3/5, Train Loss: 0.7900, Valid Loss: 0.5313
Epoch 4/5, Train Loss: 0.3639, Valid Loss: 0.4784
Epoch 5/5, Train Loss: 0.3073, Valid Loss: 0.3302
Epoch 1/5, Train Loss: 1.3347, Valid Loss: 0.3531
Epoch 2/5, Train Loss: 0.4401, Valid Loss: 0.1783
Epoch 3/5, Train Loss: 0.3300, Valid Loss: 0.1782
Epoch 4/5, Train Loss: 0.3117, Valid Loss: 0.1848
Epoch 5/5, Train Loss: 0.2498, Valid Loss: 0.1889
Epoch 1/5, Train Loss: 0.7211, Valid Loss: 0.4557
Epoch 2/5, Train Loss: 0.4285, Valid Loss: 0.2940
Epoch 3/5, Train Loss: 0.3237, Valid Loss: 0.1972
Epoch 4/5, Train Loss: 0.2703, Valid Loss: 0.3363
Epoch 5/5, Train Loss: 0.3330, Valid Loss: 0.1785
Test Accuracies. Model s: 0.9302 | Model m: 0.9302 | Model b: 0.9651


¿A qué crees que se deben las diferencias?

El modelo grande, al tener más capas y parámetros, puede aprender patrones más complejos y logra un mejor rendimiento.

## **[Ejercicio 5]**

Entrena un modelo MLP para el nuevo dataset `Iris`.

Para ello, primero crea un `Dataset` y crea también sus dataloaders:

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Verificar si CUDA está disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Cargar el conjunto de datos Iris
iris = load_iris()
data, labels = itris.data, iris.target

# Convertir a tensores
class IrisDataset(Dataset):
    def __init__(self, data, labels):
        """
        Constructor que toma un dataset y lo convierte a tensores.
        """
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Dividir el dataset en entrenamiento (25%), validación (25%) y prueba (50%)
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.5, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Crear los DataLoaders
batch_size = 32

train_ds = IrisDataset(X_train, y_train)
valid_ds = IrisDataset(X_valid, y_valid)
test_ds = IrisDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_loader =  DataLoader(valid_ds, batch_size=batch_size, shuffle=False)
test_loader =  DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Para confirmar la división
print(f"Training set size: {len(train_loader.dataset)}")
print(f"Validation set size: {len(valid_loader.dataset)}")
print(f"Test set size: {len(test_loader.dataset)}")

Training set size: 75
Validation set size: 37
Test set size: 38


Crea el modelo MLP que consideres:

In [40]:
# Definir el modelo MLP
class MLP_iris(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP_iris, self).__init__()
        ########################### COMPLETAR ###########################
        self.F1 = nn.Linear(input_dim, 64)
        self.Act1 = nn.ReLU()
        self.F2 = nn.Linear(64, 64)
        self.Act2 = nn.ReLU()
        self.F3 = nn.Linear(64, 32)
        self.Act3 = nn.ReLU()
        self.F4 = nn.Linear(32, output_dim)

        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        x = self.F1(x)
        x = self.Act1(x)
        x = self.F2(x)
        x = self.Act2(x)
        x = self.F3(x)
        x = self.Act3(x)
        x = self.F4(x)
        return x
        ######################### FIN COMPLETAR #########################


Crea (adapta) las funciones de `train_model` y `evaluate_model`:

In [43]:
# Función de entrenamiento y validación
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, n_epochs=n_epochs, device=device):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a `device`
        # Completar
    model = model.to(device)
    for epoch in range(n_epochs):
        # Modelo en modo entrenamiento
           # Completar
        model.train()
        train_loss = 0.0
        for features, labels in train_loader:

            # Enviar datos a 'device
            features = features.to(device)
            labels =  labels.to(device)

            # Forward
            outputs = model(features)
            loss =  loss_fn(outputs, labels)

            # Backward y optimización
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_loss += loss.item() * features.size(0)

        train_loss /= len(train_loader.dataset)

        # Modelo en modo validación
            # Completar
        model.eval()
        valid_loss = 0.0
        valid_loss = 0.0
        with torch.no_grad():
            for features, labels in valid_loader:
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)
                loss = loss_fn(outputs, labels)
                valid_loss += loss.item() * features.size(0)
        valid_loss /= len(valid_loader.dataset)
        if epoch % 50 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
    ######################### FIN COMPLETAR #########################

# Función para evaluar el modelo
def evaluate_model(model, test_loader, device=device, name=None):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a `device`
        # Completar
    model = model.to(device)
    # Modelo en modo validación
        # Completar
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # No hace falta calcular gradientes en evaluación
        for features, labels in test_loader:
            # Enviar datos a 'device'
            features = features.to(device)
            labels = labels.to(device)

            # Predicciones
            outputs = model(features)
            probabilities = F.softmax(outputs, dim=1)  # Ojo con la última función del modelo MLP

            # Obtener las clases predichas (devuelve la clase con mayor probabilidad)
            predicted = torch.argmax(probabilities, dim=1)

            # Contabilizar las predicciones correctas
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"{name} Accuracy: {accuracy:.4f}")

    ######################### FIN COMPLETAR #########################


Entrena solo 20 épocas y valida el modelo:

In [56]:
########################### COMPLETAR ###########################
# Inicializar el modelo
input_dim = 4
output_dim = 3
model = MLP_iris(input_dim, output_dim).to(device)

# Definir la función de pérdida y el optimizador
loss_fn = nn.CrossEntropyLoss()  #  clasificación multiclase
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 20  # Definir el número de épocas de entrenamiento

######################### FIN COMPLETAR #########################

train_model(model, train_loader, valid_loader, loss_fn, optimizer, n_epochs=n_epochs, device=device)

evaluate_model(model, train_loader, device=device, name='Train')
evaluate_model(model, valid_loader, device=device, name='Valid')
evaluate_model(model, test_loader, device=device, name='Test')

Epoch 1/20, Train Loss: 1.1000, Valid Loss: 1.0856
Train Accuracy: 0.6933
Valid Accuracy: 0.7297
Test Accuracy: 0.7632


Ejecuta varias veces la celda anterior y explica qué está ocurriendo.

Vemos que el accuracy tanto del train, valid como del test cambia según lo ejecutamos.

Estos se debe porbablemente a que pytorch inicaliza sus pesos con una distrubución aleatoria, además de que hay una aletoriedad en el orden de los batches y hemos usado Adam que es una optimización estocástica.